### GIL锁

熟悉python的都知道，在C语言写的python解释器中存在全局解释器锁，由于全局解释器锁的存在，在同一时间内，python解释器只能运行一个线程的代码，这大大影响了python多线程的性能。而这个解释器锁由于历史原因，现在几乎无法消除。 
  
python GIL 之所以会影响多线程等性能，是因为在多线程的情况下，只有当线程获得了一个全局锁的时候，那么该线程的代码才能运行，而全局锁只有一个，所以使用python多线程，在同一时刻也只有一个线程在运行，因此在即使在多核的情况下也只能发挥出单核的性能。

上述内容引用自链接：https://www.jianshu.com/p/c75ed8a6e9af  


In [1]:
from concurrent.futures import ThreadPoolExecutor

In [2]:
N = 4

In [3]:
pool = ThreadPoolExecutor(max_workers=N)

### 1. IO密集型操作
一个线程执行IO密集型操作的时候，CPU处于闲置状态，因此GIL会被释放给其他线程，从而缩短了总体的等待运行时间。

In [4]:
from time import sleep, time

In [5]:
def sleep_n_seconds(n=1):
    sleep(n)

In [6]:
def serial_func(fn, n_times=N):
    for _ in range(n_times):
        fn()
    return True

In [7]:
def concurrent_func(fn, n_times=N):
    list(pool.map(lambda x: fn(), range(n_times)))
    return True

In [8]:
def time_it(fn, way):
    assert way in ("serial", "concurrent"), "参数way必须是'serial'或者'concurrent'！"
    f = {"serial":serial_func, "concurrent": concurrent_func}[way]
    start = time()
    f(fn)
    print("运行时间为 %.3f 秒!" % (time() - start))

In [9]:
time_it(sleep_n_seconds, "serial")

运行时间为 4.004 秒!


In [10]:
time_it(sleep_n_seconds, "concurrent")

运行时间为 1.007 秒!


### 2. CPU密集型操作
一个线程执行CPU密集型操作的时候，CPU处于忙碌状态，运行1000个字节码之后GIL会被释放给其他线程，加上切换线程的时间反而会比串行代码更慢。

In [11]:
def add_one(n_times=10**7):
    ret = 0
    while ret < n_times:
        ret += 1
    return ret

In [12]:
time_it(add_one, "serial")

运行时间为 2.223 秒!


In [13]:
time_it(add_one, "concurrent")

运行时间为 2.749 秒!


### 3. 绕过GIL锁
著名的Cython模块提供了绕过GIL锁的方法，'with nogil'。我们可以用Cython模块重新编写刚才的'add_one'函数，再用python包装为python可调用的函数。代码如下：

In [14]:
%load_ext Cython

In [26]:
%%cython
def _add_one(long n_times= 10 ** 7):
    cdef long ret = 0
    with nogil:
        while ret < n_times:
            ret += 1
    return ret

def add_one(n_times=10 ** 7):
    return _add_one(n_times)

In [31]:
time_it(add_one, "serial")

运行时间为 0.038 秒!


In [32]:
time_it(add_one, "concurrent")

运行时间为 0.023 秒!
